# Deploying dotjson - Deepseek-R1-Distill-Qwen-32B with structured outputs from AWS Marketplace 


## Introduction
This sample notebook shows you how to deploy [dotjson - Deepseek-R1-Distill-Qwen-32B with structured outputs](https://aws.amazon.com/marketplace/pp/prodview-mb7desoblqh2w) using Amazon SageMaker.

Make your AI-powered applications reliable with our structured outputs solution for DeepSeek-R1-Distill-Qwen-32B. This fully managed service seamlessly integrates with your AWS infrastructure, delivering guaranteed JSON Schema-compliant outputs directly from the language model. Simply deploy through AWS Marketplace, and receive valid JSON responses every time. Our solution leverages proprietary optimization technology designed specifically for AWS environments, ensuring zero-latency structured outputs. Unlike traditional LLM implementations that require complex post-processing and development overhead, our service dramatically increases schema compliance while maintaining the model's full performance. Transform unreliable AI outputs into production-ready, structured data without sacrificing speed or accuracy.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.


## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [dotjson - Deepseek-R1-Distill-Qwen-32B with structured outputs](https://aws.amazon.com/marketplace/pp/prodview-mb7desoblqh2w) 
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.
2. Choose or create an execution role that has the **AmazonSageMakerFullAccess** IAM policy attached. If it already exists, the execution role can be found below the **Product Arn** on the configuration page. Otherwise, go to the **IAM** console on AWS to create a role with Sagemaker execution access.

In [ ]:
model_package_arn = (
    "<Customer to specify Model package ARN corresponding to their AWS region>"
)


execution_role = "<Customer to specify Model package ARN with AmazonSageMakerFullAccess IAM policy attached>"

In [ ]:
import json
import boto3
import sagemaker

In [ ]:
session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=session)
sagemaker_client = sagemaker_session.sagemaker_client

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

**Define the endpoint configuration:**

In [ ]:
model_name = "dotjson-with-DeepSeek-R1-Distill-Qwen-32B"
content_type = "application/json"
inference_instance_type = "ml.g6e.12xlarge"
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-notebook"

### A. Create an endpoint

In [ ]:
# Deploy a model on sagemaker using the model package arn 
create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    PrimaryContainer={"ModelPackageName": model_package_arn},
    ExecutionRoleArn=execution_role,
    EnableNetworkIsolation=True,
)

# Create endpoint configuration
endpoint_config_name = f"{model_name}-config"
create_endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "InitialInstanceCount": 1,
            "ModelName": model_name,
            "InstanceType": inference_instance_type,
        }
    ],
)
print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

# Create endpoint
sagemaker_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

In [ ]:
with open("payload.json", "r") as f:
    payload = f.read()

### C. Perform real-time inference

In [ ]:
# Invoke the endpoint and extract the response body
runtime = session.client("sagemaker-runtime")
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name",
    ContentType=content_type,
    Accept=content_type,
    Body=payload,
)
body = json.loads(response["Body"].read().decode("utf-8"))

### D. Visualize output

In [ ]:
# Save model response to out file
output_file_name = "character.json"
with open(output_file_name, "w") as outfile:
    json.dump(body, outfile, indent=4)

# View the structured output produced by the model
msg = body["choices"][0]["message"]
content = msg["content"]
character = json.loads(content)
character

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

### B. Delete the model

In [ ]:
sagemaker_client.delete_model(ModelName=model_name)

### C. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

